In [180]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [181]:
data = pd.read_csv('Tweets_dl.csv',sep=',')
data.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [182]:
data.shape

(19138, 15)

In [183]:
data.replace(np.nan, 'negative')
data['airline_sentiment'] = data['airline_sentiment'].replace(np.nan, 'negative')
data['text'] = data['text'].replace(np.nan, 'negative')

In [165]:
#to check

In [173]:
'''sentiment_map={0:'negative',1:'positive'}
data.insert(2,'label_int',[sentiment_map[s] for s in data.label],True)
#data['sentiment_int']=[sentiment_map[s] for s in data.sentiment]
data.head()'''

"sentiment_map={0:'negative',1:'positive'}\ndata.insert(2,'label_int',[sentiment_map[s] for s in data.label],True)\n#data['sentiment_int']=[sentiment_map[s] for s in data.sentiment]\ndata.head()"

In [184]:
data['text'].loc[2001]

'@united flight #1 no luck on #standby'

In [185]:
data['airline_sentiment'].loc[1]

'positive'

In [187]:
reviews = np.array(data['text'])[:]
labels = np.array(data['airline_sentiment'])[:]

In [188]:
from collections import Counter

Counter(labels)

Counter({'neutral': 3778, 'positive': 2887, 'negative': 12473})

In [189]:
data.isnull().sum()

tweet_id                            0
airline_sentiment                   0
airline_sentiment_confidence        0
negativereason                   6665
negativereason_confidence        4964
airline                             0
airline_sentiment_gold          19081
name                                0
negativereason_gold             19093
retweet_count                       0
text                                0
tweet_coord                     17840
tweet_created                       0
tweet_location                   6263
user_timezone                    6416
dtype: int64

# DATA PRE_PROCESSING

In [190]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import RegexpTokenizer
import csv

def preProcessor(text):
    import re
    from string import punctuation
    textt=re.sub(r'(http|ftp|https):\/\/([\w\-_]+(?:(?:\.[\w\-_]+)+))([\w\-\.,@?^=%&:/~\+#]*[\w\-\@?^=%&/~\+#])?', ' ', text)
    textt=re.sub(r'['+punctuation+']',' ',text)
    textt=re.sub(r'#(\w+)',' ',text)
    textt=re.sub(r'@(\w+)',' ',text)
    #print(token.tokenize(text))
    return text

token=RegexpTokenizer(r'\w+')
cv=CountVectorizer(lowercase=True,preprocessor=preProcessor,stop_words='english',ngram_range=(1,1),tokenizer=token.tokenize)
#text_counts=cv.fit_transform(data['Tweet'])
text_counts=cv.fit_transform(data['text'].values.astype('U'))

In [191]:
from sklearn.model_selection import train_test_split
# x_train, x_test, y_train, y_test = train_test_split(text_counts,data['sentiment'],test_size=0.3)
x_train, x_test, y_train, y_test = train_test_split(text_counts,data['airline_sentiment'],test_size=0.3)

In [192]:
from sklearn.naive_bayes import *
from sklearn import metrics

clf=MultinomialNB()
#clf_gaus=GaussianNB()
#clf_ber=BernoulliNB()
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.7836990595611285

In [193]:
#Ber_NB
clf=BernoulliNB()
clf.fit(x_train,y_train)
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.7427725531173807

In [194]:
#svm
from sklearn.svm import LinearSVC

In [195]:
#linear
clf=LinearSVC()
clf.fit(x_train,y_train)
clf.fit(x_train,y_train)
pred=clf.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.829153605015674

In [196]:
# Polynomial Kernel
from sklearn.svm import SVC
svclassifier = SVC(kernel='poly', degree=8)
svclassifier.fit(x_train, y_train)
pred=svclassifier.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.6863462208289794

In [197]:
#Gaussian Kernel
svclassifier = SVC(kernel='rbf')
svclassifier.fit(x_train, y_train)
pred=svclassifier.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.8157436433298503

In [198]:
#Sigmoid Kernel
svclassifier = SVC(kernel='sigmoid')
svclassifier.fit(x_train, y_train)
pred=svclassifier.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.7464298153953326

In [199]:
# K=5
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(x_train, y_train)
pred=knn.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.5990943921978404

In [200]:
# K=7
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=7)
knn.fit(x_train, y_train)
pred=knn.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.5724486241727621

In [201]:
# K=4
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=4)
knn.fit(x_train, y_train)
pred=knn.predict(x_test)
metrics.accuracy_score(y_test, pred)

0.6057122953674677